# Código do TCC de Paulo de Tarso, da EMAp FGV

In [1]:
import nltk
#Se for preciso, executar o comando abaixo
#nltk.download()
import unicodedata
import string
import os
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import CountVectorizer as CV
#Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

## Códigos auxiliares
### Código para remover símbolos das strings

In [2]:
def clean_string(s):
    s = s.replace("\n"," ")
    return ''.join(x for x in unicodedata.normalize('NFKD', s)
                   if x in string.ascii_letters + " ").lower()

## Parâmetros

In [12]:
#Parâmetros
inicio_atas = 104 #Utilizarei as atas a partir de 2005
final_atas = 206

total_atas = final_atas - inicio_atas + 1
atas = range(inicio_atas,final_atas+1)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

## Tokenização dos textos

### Obter textos dos arquivos

In [13]:
texts = []

#Ir para a pasta com os arquivos
os.chdir(path)

#Obter o texto dos arquivos
for n in atas:
    file = open("COPOM_" + str(n) + ".txt")
    texts.append(clean_string(file.read()))
    file.close()
    
#Voltar uma pasta
os.chdir("..")

### Vetorizar os textos obtidos

#### Stopwords

In [14]:
sw = [clean_string(word) for word in 
             open("stopwords.txt",encoding="utf-8").read().splitlines()[1:]]

#### Vectorização

In [47]:
vectorizer = CV(stop_words=sw,min_df=50)
arrays = vectorizer.fit_transform(texts).toarray()
#d_arrays = dict(zip(range(inicio_atas,final_atas+1),arrays))

## Obter os valores das variações na meta da Selic em cada reunião do Copom

In [49]:
file = open("Cortes.txt")
cortes = [int(line.split(";")[2]) for line in file.read().splitlines()[1:]]
cortes = [int(n/abs(n)) if n != 0 else 0 for n in cortes]
cortes.reverse()
file.close()

## Métodos de classificação
Me basearei nos modelos contidos na biblioteca Scikit Learn. Os modelos estão em http://scikit-learn.org/stable/supervised_learning.html.

#### Função que recebe o modelo desejado e retorna o score médio e o valor predito médio para a última reunião (não presente na amostra)

In [157]:
def RunModel(model,n_executions):
    classifier = model
    score = 0
    last_prediction = 0
    for i in range(n_executions):
        
        #Separação em dados para treino e dados para teste do modelo
        #(Aqui, utilizo até a antepenúltima reunião; a penúltima será
        #utilizada para prever o resultado da última).
        X_train,X_test,y_train,y_test = tts(arrays[:-2],cortes[:-1],train_size=0.9)

        #Treino do modelo nos dados
        classifier.fit(X_train,y_train)

        #Resultados
        score += classifier.score(X_test,y_test)
        last_prediction += classifier.predict(arrays[-2])[0]
        
    score /= n_executions
    last_prediction /= n_executions
    return "Score: " + str(score) + " Predição da última reunião: " + str(last_prediction)

In [142]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.kernel_ridge import KernelRidge as KRR
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier as SGDC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.naive_bayes import BernoulliNB as BNB
from sklearn.tree import DecisionTreeClassifier as DTC

models = [LR,LDA,QDA,KRR,SVC,SGDC,KNC,GPC,PLSR,GNB,MNB,BNB,DTC]
names = ["Generalized Linear Model","Linear Discriminant Analysis",
        "Quadratic Discriminant Analysis","Kernel Ridge Regression",
        "Support Vector Machine Classifier","Stochastic Gradient Descent",
        "K Nearest Neighbors","Gaussian Process",
         "Partial Least Squares Regressors","Gaussian Naive Bayes",
         "Multinomial Naive Bayes","Bernoulli Naive Bayes",
         "Decision Tree Classifier"]

In [148]:
for i in range(len(models)):
    print(names[i])
    print(RunModel(models[i],1000))

Generalized Linear Model
Score: 0.531146306319 Predição da última reunião: -0.826825723391
Linear Discriminant Analysis
Score: 0.757909090909 Predição da última reunião: -0.974
Quadratic Discriminant Analysis
Score: 0.392454545455 Predição da última reunião: -0.526
Kernel Ridge Regression
Score: 0.516387914713 Predição da última reunião: -0.842941519853
Support Vector Machine Classifier
Score: 0.384272727273 Predição da última reunião: -1.0
Stochastic Gradient Descent
Score: 0.543272727273 Predição da última reunião: -0.901
K Nearest Neighbors
Score: 0.417363636364 Predição da última reunião: -0.997
Gaussian Process
Score: 0.312545454545 Predição da última reunião: 1.0
Partial Least Squares Regressors
Score: -0.0167780419141 Predição da última reunião: [-2.68361038]
Gaussian Naive Bayes
Score: 0.635272727273 Predição da última reunião: -0.999
Multinomial Naive Bayes
Score: 0.538818181818 Predição da última reunião: 0.0
Bernoulli Naive Bayes
Score: 0.484818181818 Predição da última reun

### Generalized Linear Model

In [158]:
from sklearn.linear_model import LinearRegression as LR
RunModel(LR(),1000)

'Score: 0.530687283602 Predição da última reunião: -0.829286163544'

### Linear Discriminant Analysis

In [121]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
RunModel(LDA(),1000)

Score: 0.768 Predição da última reunião: -0.969


### Quadratic Discriminant Analysis

In [122]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
RunModel(QDA(),1000)

Score: 0.384090909091 Predição da última reunião: -0.558


### Kernel Ridge Regression

In [123]:
from sklearn.kernel_ridge import KernelRidge as KRR
RunModel(KR(),10000)

Score: 0.526121517809 Predição da última reunião: -0.840371189118


### Support Vector Machine

In [124]:
from sklearn.svm import SVC
RunModel(SVC(),100)

Score: 0.374545454545 Predição da última reunião: -1.0


### Stochastic Gradient Descent

In [125]:
from sklearn.linear_model import SGDClassifier as SGDC
RunModel(SGDC(),1000)

Score: 0.539727272727 Predição da última reunião: -0.918


### Nearest Neighbors

In [126]:
from sklearn.neighbors import KNeighborsClassifier as KNC
RunModel(KNC(),1000)

Score: 0.413181818182 Predição da última reunião: -0.991


### Gaussian Process

In [127]:
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
RunModel(GPC(),1000)

Score: 0.322090909091 Predição da última reunião: 1.0


### Cross Decomposition

In [131]:
from sklearn.cross_decomposition import PLSRegression as PLSR
RunModel(PLSR(),1000)

Score: 0.0218190452938 Predição da última reunião: [-2.66567714]


### Gaussian Naive Bayes

In [137]:
from sklearn.naive_bayes import GaussianNB as GNB
RunModel(GNB(),1000)

Score: 0.637636363636 Predição da última reunião: -0.998


### Multinomial Naive Bayes

In [138]:
from sklearn.naive_bayes import MultinomialNB as MNB
RunModel(MNB(),1000)

Score: 0.534909090909 Predição da última reunião: 0.0


### Gaussian Naive Bayes

In [139]:
from sklearn.naive_bayes import BernoulliNB as BNB
RunModel(BNB(),1000)

Score: 0.483454545455 Predição da última reunião: -0.759


### Decision Tree Classifier

In [149]:
from sklearn.tree import DecisionTreeClassifier as DTC
RunModel(DTC(),1000)

'Score: 0.616909090909 Predição da última reunião: -0.729'

### Bagging meta-estimator

In [152]:
from sklearn.ensemble import BaggingClassifier as BC
RunModel(BC(),100)

'Score: 0.674545454545 Predição da última reunião: -0.86'

### Random Forest Classifier

In [154]:
from sklearn.ensemble import RandomForestClassifier as RFC
RunModel(RFC(),1000)

'Score: 0.679363636364 Predição da última reunião: -0.971'

### Extremely Randomized Trees

In [156]:
from sklearn.ensemble import ExtraTreesClassifier as ETC
RunModel(ETC(),1000)

'Score: 0.716090909091 Predição da última reunião: -0.982'